In [7]:
import pandas as pd
import numpy as np

from scipy import stats

df = pd.concat( [ pd.read_csv(f"data_collected_{i}.csv") for i in ["red","blue"] ] )
df.head()

,click,visit,group
0,0,1,control
1,1,1,control
2,0,1,control
3,1,1,control
4,0,1,control


In [62]:
df1 = df.copy()
for i in ['visit','click']:
    df1[i] = df1[i].astype(int)
df1 = df1.reset_index().rename( columns={'index':'day'} )
df1 = df1.pivot_table( index='day', columns='group', values=df1.select_dtypes(exclude='object').columns ,aggfunc='sum')
df1 = df1.swaplevel( axis=1 )
df1 = df1.reindex(sorted(df1.columns), axis=1).fillna(0)
df1.columns = ["_".join(i) for i in df1.columns]
for i in df1.columns:
    df1[f"acc_{i}"] = df1[i].cumsum()
df1 = df1.rename(columns={
    'control_click':"clicks_B",
    'treatment_click':"clicks_A",
    "control_visit":"visits_B",
    "treatment_visit":"visits_A",
    "acc_control_click":"acc_clicks_B",
    "acc_treatment_click":"acc_clicks_A",
    "acc_control_visit":"acc_visits_B",
    "acc_treatment_visit":"acc_visits_A"
})
df1.head()

,clicks_B,visits_B,clicks_A,visits_A,acc_clicks_B,acc_visits_B,acc_clicks_A,acc_visits_A
day,,,,,,,,
0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0
1,2.0,2.0,0.0,0.0,2.0,4.0,0.0,0.0
2,0.0,2.0,0.0,0.0,2.0,6.0,0.0,0.0
3,2.0,2.0,0.0,0.0,4.0,8.0,0.0,0.0
4,0.0,2.0,0.0,0.0,4.0,10.0,0.0,0.0


In [63]:
N_mc = 100000

## Mean and variance of groups generations

In [64]:
len(df1)

568

In [65]:
proba_b_better_a = []

for day in range( len ( df1 )-560 ):

    u_a , var_a = stats.beta.stats( a = 1 + df1.loc[day, 'acc_clicks_A'], \
                                b = 1 + (df1.loc[day, 'acc_visits_A'] - df1.loc[day, 'acc_clicks_A']), moments='mv')

    u_b , var_b = stats.beta.stats( a = 1 + df1.loc[day, 'clicks_B'], \
                                b = 1 + (df1.loc[day, 'visits_B'] - df1.loc[day, 'clicks_B']), moments='mv')

    print( u_a )

    # Normal distribution Sample A
    x_a = np.random.normal( loc = u_a,
                        scale = 1.25*np.sqrt( var_a ) ,
                        size = N_mc)

    # Normal distribution Sample B
    x_b = np.random.normal( loc = u_b,
                        scale = 1.25*np.sqrt( var_b ) ,
                        size = N_mc)

    print( x_a )

    # Norm distribution function of page A
    fa = stats.beta.pdf ( x_a,
                a = 1 + df1.loc[day, 'acc_clicks_A'],
                b = 1 + ( df1.loc[day, 'acc_visits_A'] - df1.loc[day, 'acc_clicks_A']
                            ))


    # Norm distribution function of page B
    fb = stats.beta.pdf ( x_b,
                a = 1 + df1.loc[day, 'clicks_B'],
                b = 1 + ( df1.loc[day, 'visits_B'] - df1.loc[day, 'clicks_B'] 
                            ))
    
    print( fa )

    # Beta distribution function of page A
    ga = stats.norm.pdf( x_a,
                loc = u_a,
                scale = 1.25*np.sqrt( var_a )
                            )

    # Beta distribution function of page B
    gb = stats.norm.pdf( x_b,
                loc = u_b,
                scale = 1.25*np.sqrt( var_b )
                            )
    
    print(ga)


    # Beta / Normal
    y = ( fa*fb ) / ( ga*gb )

    yb = y[x_b >= x_a]

    p = ( 1/ N_mc ) * np.sum(yb)

    proba_b_better_a.append(p)

0.5
[ 0.87883237  0.47394577  1.00090325 ...  0.53383862  0.9651467
 -0.17578023]
[1. 1. 0. ... 1. 1. 0.]
[0.6371677  1.10270313 0.42185215 ... 1.10073071 0.48168894 0.19142182]
0.5
[ 0.45670835  0.36684222  1.13142298 ...  0.84873059  0.57957703
 -0.07225161]
[1. 1. 0. ... 1. 1. 0.]
[1.09765319 1.03281101 0.2391612  ... 0.69307103 1.0790213  0.31439005]
0.5
[ 0.12129711 -0.12275385  0.18872804 ...  0.56139611  1.12555967
  0.14803278]
[1. 0. 1. ... 1. 0. 1.]
[0.63740774 0.24935777 0.76209256 ... 1.08969346 0.24602641 0.68706147]
0.5
[ 0.72049601  0.20522849  0.28720999 ... -0.18811729  0.80189876
  0.27158841]
[1. 1. 1. ... 0. 1. 1.]
[0.9172972  0.79192602 0.92913393 ... 0.17944446 0.77909885 0.90486576]
0.5
[0.77798074 0.87583385 0.17937234 ... 0.36642558 0.30756712 0.74100347]
[1. 1. 1. ... 1. 1. 1.]
[0.82171784 0.64272848 0.74498679 ... 1.03237036 0.95903708 0.88455886]
0.5
[0.35910481 0.64598639 0.07534383 ... 0.61918218 0.70125325 0.69653536]
[1. 1. 1. ... 1. 1. 1.]
[1.02443543 1

In [48]:
fig, axs = 

0.24918409601076405